In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
print(keras.__version__)

2.1.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1
2.2.4-tf


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

In [3]:

training_data = np.load('training_data.npy', allow_pickle=True)
x_train = []
y_train = []

for x in range(len(training_data)):
    x_train.append(training_data[x][0])
    y_train.append(training_data[x][1][:3])
    
x_train = np.array(x_train, dtype=np.float16)
y_train = np.array(y_train, dtype=np.float16)



In [4]:
print(x_train[0][0][0][0])
for a in range(len(x_train)):
    for b in range(len(x_train[a])):
        for c in range(len(x_train[a][b])):
                x_train[a][b][c]  =  x_train[a][b][c] / 255

print(x_train[0][0][0][0])

[255. 255. 255.]
[1. 1. 1.]


In [9]:
from keras.models import Sequential
from keras.layers import TimeDistributed, LSTM, Flatten, Dense, InputLayer, MaxPooling2D, Dropout, Activation, Embedding, GRU, ConvLSTM2D
from keras.layers.convolutional import Convolution2D
from keras import optimizers
from keras.models import load_model
from keras import initializers

model = Sequential()


model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last'), input_shape=(3, 150, 200, 3)))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)

model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last')))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)

model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.25)))
print(model.output_shape)

model.add(TimeDistributed(Convolution2D(16, (3,3), data_format='channels_last')))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)


model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.25)))
print(model.output_shape)

model.add(TimeDistributed(Flatten()))
print(model.output_shape)

model.add(LSTM(256, kernel_initializer=initializers.RandomNormal(stddev=0.001))) #128
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Dense(100))
print(model.output_shape)

model.add(Dense(80))
print(model.output_shape)

model.add(Dense(40))
print(model.output_shape)

model.add(Dense(3, activation='sigmoid'))
print(model.output_shape)

model.summary()


(None, 3, 147, 197, 32)
(None, 3, 144, 194, 32)
(None, 3, 28, 38, 32)
(None, 3, 26, 36, 16)
(None, 3, 5, 7, 16)
(None, 3, 560)
(None, 256)
(None, 100)
(None, 80)
(None, 40)
(None, 3)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_12 (TimeDis (None, 3, 147, 197, 32)   1568      
_________________________________________________________________
time_distributed_13 (TimeDis (None, 3, 147, 197, 32)   0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 3, 144, 194, 32)   16416     
_________________________________________________________________
time_distributed_15 (TimeDis (None, 3, 144, 194, 32)   0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 3, 28, 38, 32)     0         
_________________________________________________________________
tim

In [10]:
opt = optimizers.rmsprop(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train,
          batch_size=10,
          epochs=10,
          verbose=1)



Epoch 1/10
7239/7239 [==============================] - 133s 18ms/step - loss: 0.1418 - accuracy: 0.6929
Epoch 2/10
7239/7239 [==============================] - 87s 12ms/step - loss: 0.0980 - accuracy: 0.8011
Epoch 3/10
7239/7239 [==============================] - 87s 12ms/step - loss: 0.0834 - accuracy: 0.8369
Epoch 4/10
7239/7239 [==============================] - 86s 12ms/step - loss: 0.0735 - accuracy: 0.8591
Epoch 5/10
7239/7239 [==============================] - 87s 12ms/step - loss: 0.0677 - accuracy: 0.8732
Epoch 6/10
7239/7239 [==============================] - 86s 12ms/step - loss: 0.0625 - accuracy: 0.8820
Epoch 7/10
7239/7239 [==============================] - 86s 12ms/step - loss: 0.0591 - accuracy: 0.8882
Epoch 8/10
7239/7239 [==============================] - 86s 12ms/step - loss: 0.0570 - accuracy: 0.8945
Epoch 9/10
7239/7239 [==============================] - 86s 12ms/step - loss: 0.0521 - accuracy: 0.9044
Epoch 10/10
7239/7239 [==============================] - 86s 12

In [12]:
model.save("prova.h5")

In [93]:
results = []
for x in range(100):
    data = np.array(x_train[x])
    data = data.reshape(-1, 3, 150, 200, 3)
    prediction = model.predict_classes(data)
    results.append(prediction)
print(results)

[array([2], dtype=int64), array([2], dtype=int64), array([5], dtype=int64), array([0], dtype=int64), array([1], dtype=int64), array([2], dtype=int64), array([3], dtype=int64), array([0], dtype=int64), array([1], dtype=int64), array([1], dtype=int64), array([1], dtype=int64), array([1], dtype=int64), array([3], dtype=int64), array([1], dtype=int64), array([0], dtype=int64), array([2], dtype=int64), array([5], dtype=int64), array([1], dtype=int64), array([1], dtype=int64), array([3], dtype=int64), array([2], dtype=int64), array([5], dtype=int64), array([5], dtype=int64), array([2], dtype=int64), array([4], dtype=int64), array([2], dtype=int64), array([0], dtype=int64), array([5], dtype=int64), array([0], dtype=int64), array([1], dtype=int64), array([4], dtype=int64), array([1], dtype=int64), array([0], dtype=int64), array([3], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), array([2], dtype=int64), array([3], dtype=int64), array([5], dtype=int64), array([0], dtype=int64),

In [13]:
prediction = model.predict_classes(data)
print(prediction)

NameError: name 'data' is not defined

In [7]:
def get_grb():
    grb = Grabber(bbox=(1,26,801,626))
    grabbedscreen = None
    grabbedscreen = grb.grab(grabbedscreen)
    img_resized = cv2.resize(grabbedscreen, (200,150))
    final_img = np.array(img_resized)
    return final_img
    

In [8]:
keyList = []
for char in "P":
    keyList.append(char)

def key_check():
    keys = []
    for key in keyList:
        if wapi.GetAsyncKeyState(ord(key)):
            keys.append(key)
    return keys

In [9]:
def key_activator(key):
    keyboard.release("w")
    keyboard.release("a")
    keyboard.release("d")
    
    if key == 0:
        keyboard.press("w")
        
    elif key == 1:
        keyboard.press("w")
        keyboard.press("a")

    elif key == 2:
        keyboard.press("w")
        keyboard.press("d")

    else:
        print("Error")

In [13]:
import time
from pynput.keyboard import Controller
from grabber import Grabber
import win32api as wapi
keyboard = Controller()
def director():
    running = True
    time.sleep(5)
    print("Starting")
    while running:
    #capture seq
        grab_seq = []
        for i in range(3):
                last_time = time.time()
                grab_seq.append(get_grb())
                if i > 2:
                    waittime = (1.0/capturerate)-(time.time()-last_time)
                    if waittime>0.0:
                        time.sleep(waittime)
        #model predict
        img_seq = np.array(grab_seq, dtype=np.float16)

        for a in range(len(img_seq)):
            for b in range(len(img_seq[a])):
                img_seq[a][b]  =  img_seq[a][b] / 255
        
        img_seq = img_seq.reshape(-1, 3, 150, 200, 3)

        prediction = model.predict_classes(img_seq)

        key_activator(prediction)
        
        keys = key_check()
        if "P" in keys:
            running = False
            keyboard.release("w")
            keyboard.release("a")
            keyboard.release("d")
            print("Stopped")
director()    

Starting
Stopped


In [ ]:
awwwwwwaaaawwawawawawawwwwwwwwwwwwwwwawawawawawawwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwawwwwwwwwwwwwwwwwawawawwwwwwwwwwwwwwdwdwdwdwdwdwdwwwwwwawwwwwwwwwwwwwwwdwdwwwwwwwwwdwdwdwwwwwwwwwwwwwwwwwwwwwwwwwdwdwdwdwdwwwwawawawawawwwwdwdwdwwwwwwwwwwwwwwwwwwwwwwwdwdwdwdwwwwwwwwwwwwwwwwwwwdwdwdwdwdwwwwwwwwwwwwwwwdwdwdwdwdwwwwwwwwwdwdwdwdwdwdwwwwwwwwwwwwwwwwwwdwawdwdwawawawawawdwdwdwdwdwdwdwdwdwawawawawawawawawawawdwdwdwdwdwdwdwdwdwawawawawawawawawawawawdwdwdwdwdwdwdwdwdwdwawawawawawawawawawawawawdwdwdwdwdwdwdwdwdwawawawawawawawawawawawdwdwdwdwdwdwdwdwdwdwdwdwdwawawawawawawawawawawawawawawawawdwdwdwdwdwdwdwdwdwdwdwdwdwdwawawawawawawawawawawawawdwdwdwdwdwdwdwdwdwdwawawawawawawawawdwdwdwdwdwdwdwdwdwdwdwawawdwawawawawawawawdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwdwapwawawawdwdwdwdwdwawawawawawawawawawawawawawawawwwwa